<a href="https://colab.research.google.com/github/RandomOgre101/dl-project/blob/main/DL_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip train_test.zip

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import numpy as np
import tensorflow as tf
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [6]:
path = './'

apple_pie_train = glob.glob(path + 'train/apple_pie/*')
apple_pie_test = glob.glob(path + 'test/apple_pie/*')

cannoli_train = glob.glob(path + 'train/cannoli/*')
cannoli_test = glob.glob(path + 'test/cannoli/*')

ramen_train = glob.glob(path + 'train/ramen/*')
ramen_test = glob.glob(path + 'test/ramen/*')

In [7]:
print("Analysis by class: \n\t\tTrain\tTest")
print(f"Apple Pie\t{len(apple_pie_train)}\t{len(apple_pie_test)}")
print(f"Cannoli\t\t{len(cannoli_train)}\t{len(cannoli_test)}")
print(f"Ramen\t\t{len(ramen_train)}\t{len(ramen_test)}")

Analysis by class: 
		Train	Test
Apple Pie	750	250
Cannoli		750	250
Ramen		750	250


In [26]:
class_to_category = {
     0 : 'apple pie',
     1 : 'cannoli',
     2 : 'ramen'
}

In [9]:
train_datagen = ImageDataGenerator(featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 rotation_range=5,
                 shear_range=0.2,
                 zoom_range=0.2,
                 width_shift_range=0.05,
                 height_shift_range=0.05,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 horizontal_flip=True,
                 vertical_flip=False,
                 rescale=1/255)
train_generator = train_datagen.flow_from_directory(
        path + 'train/',
        target_size=(224,224),
        class_mode='categorical',
        batch_size=32)

test_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = test_datagen.flow_from_directory(
        path + 'test/',
        target_size=(224,224),
        class_mode='categorical',
        batch_size=32)

Found 2250 images belonging to 3 classes.
Found 750 images belonging to 3 classes.


In [10]:
def create_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(3, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

def ensemble(models, model_input):
    outputs = [model(model_input) for model in models]
    avg = tf.keras.layers.average(outputs)
    model = Model(inputs=model_input, outputs=avg, name='ensemble')
    return model

In [11]:
base_model1 = VGG16(weights='imagenet', include_top=False)
model1 = create_model(base_model1)

base_model2 = ResNet50(weights='imagenet', include_top=False)
model2 = create_model(base_model2)

base_model3 = DenseNet121(weights='imagenet', include_top=False)
model3 = create_model(base_model3)

29084464/29084464 [==============================] - 0s 0us/step


In [12]:
model1.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model2.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model3.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model1.fit(train_generator, validation_data=validation_generator, epochs=10)
model2.fit(train_generator, validation_data=validation_generator, epochs=10)
model3.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10
71/71 [==============================] - 75s 720ms/step - loss: 0.9885 - accuracy: 0.4782 - val_loss: 0.6561 - val_accuracy: 0.6987
Epoch 2/10
71/71 [==============================] - 45s 631ms/step - loss: 0.5601 - accuracy: 0.7609 - val_loss: 0.8020 - val_accuracy: 0.6653
Epoch 3/10
71/71 [==============================] - 47s 657ms/step - loss: 0.4794 - accuracy: 0.7982 - val_loss: 0.4809 - val_accuracy: 0.7800
Epoch 4/10
71/71 [==============================] - 45s 627ms/step - loss: 0.3795 - accuracy: 0.8436 - val_loss: 0.4109 - val_accuracy: 0.8187
Epoch 5/10
71/71 [==============================] - 44s 624ms/step - loss: 0.3447 - accuracy: 0.8627 - val_loss: 0.3763 - val_accuracy: 0.8333
Epoch 6/10
71/71 [==============================] - 45s 628ms/step - loss: 0.3030 - accuracy: 0.8822 - val_loss: 0.2806 - val_accuracy: 0.8800
Epoch 7/10
71/71 [==============================] - 45s 627ms/step - loss: 0.2414 - accuracy: 0.9133 - val_loss: 0.2534 - val_accuracy: 0.8933

In [14]:
models = [model1, model2, model3]
model_input = tf.keras.layers.Input(shape=(224, 224, 3))
ensemble_model = ensemble(models, model_input)

In [16]:
model1.save_weights('model1_weights.h5')
model2.save_weights('model2_weights.h5')
model3.save_weights('model3_weights.h5')

In [18]:
ensemble_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
ensemble_model.evaluate(validation_generator)

24/24 [==============================] - 13s 291ms/step - loss: 0.2807 - accuracy: 0.9493


[0.2806558907032013, 0.9493333101272583]

In [21]:
from keras.preprocessing import image

image_path = 'apple pie.jpg'
img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

In [22]:
prediction = ensemble_model.predict(img)

1/1 [==============================] - 6s 6s/step


In [23]:
prediction

array([[0.6666667 , 0.        , 0.33333334]], dtype=float32)

In [28]:
class_to_category[int(np.argmax(prediction, axis=1))]

'apple pie'

In [29]:
image_path2 = 'cannoli.jpg'
img2 = image.load_img(image_path2, target_size=(224, 224))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2, axis=0)

In [31]:
prediction2 = ensemble_model.predict(img2)

1/1 [==============================] - 0s 35ms/step


In [32]:
class_to_category[int(np.argmax(prediction2, axis=1))]

'cannoli'